## 난임 환자 대상 임신 성공 여부 예측

### LGAimers 6th 온라인 해커톤

Import

In [1]:
import pandas as pd
import lightgbm as lgb
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

### Data Load

In [2]:
# 데이터 로드
DI_train = pd.read_csv('../data/DI_train_dataset_31.csv')
DI_test = pd.read_csv('../data/DI_test_dataset_31.csv')

In [3]:
# ID 열을 제외한 특성과 타겟 변수 분리
DI_X = DI_train.drop(['임신_성공_여부', 'ID'], axis=1)
DI_y = DI_train['임신_성공_여부']

In [4]:
print(f"DI_X shape: {DI_X.shape}")
print(f"DI_test shape: {DI_test.drop('ID', axis=1).shape}")

DI_X shape: (6290, 40)
DI_test shape: (2176, 40)


### 인코딩 

In [5]:
DI_categorical_columns = [
    "시술_당시_나이",
    "정자_기증자_나이"
]

In [6]:
# 모든 범주형 변수를 문자열로 변환
DI_X[DI_categorical_columns] = DI_X[DI_categorical_columns].astype(str)
DI_test[DI_categorical_columns] = DI_test[DI_categorical_columns].astype(str)

# OrdinalEncoder를 사용하여 범주형 변수 인코딩
DI_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

DI_X[DI_categorical_columns] = DI_encoder.fit_transform(DI_X[DI_categorical_columns])
DI_test[DI_categorical_columns] = DI_encoder.transform(DI_test[DI_categorical_columns])

## Modeling

In [7]:
# 데이터 분할
DI_X_train, DI_X_test, DI_y_train, DI_y_test = train_test_split(DI_X, DI_y, test_size=0.2, random_state=42)

### DI 데이터

In [8]:
import optuna
import lightgbm as lgb
from sklearn.metrics import roc_auc_score

# 목적 함수 정의
def objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 1000, 8000),
        'max_depth': trial.suggest_int('max_depth', 3, 300),
        'learning_rate': trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 2, 2048),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 100),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-3, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-3, 10.0, log=True),
        
        'boosting_type' : 'gbdt',
        'random_state': 42,
        'objective': 'binary',
        'metric': 'auc',
        'verbose': -1   
    }

    model = lgb.LGBMClassifier(**param)
    model.fit(DI_X_train, DI_y_train)
    
    y_pred_proba = model.predict_proba(DI_X_test)[:, 1]
    
    auc = roc_auc_score(DI_y_test, y_pred_proba)
    return auc

# Optuna 스터디 생성 및 최적화 실행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=4000)

# 최적의 하이퍼파라미터 출력
print("Best trial:")
trial = study.best_trial
print(f"  Value: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2025-02-16 01:06:52,622] A new study created in memory with name: no-name-57abcf98-0a40-448e-a63a-eece9e4be9c4
[I 2025-02-16 01:06:59,748] Trial 0 finished with value: 0.6749699650502403 and parameters: {'n_estimators': 3214, 'max_depth': 195, 'learning_rate': 4.0144545471407e-05, 'num_leaves': 145, 'min_child_samples': 38, 'subsample': 0.8539581268518178, 'reg_alpha': 0.01479138621079779, 'reg_lambda': 0.009327682147317468}. Best is trial 0 with value: 0.6749699650502403.
[I 2025-02-16 01:07:01,369] Trial 1 finished with value: 0.6590869375273045 and parameters: {'n_estimators': 3894, 'max_depth': 45, 'learning_rate': 0.04312177769131268, 'num_leaves': 1257, 'min_child_samples': 37, 'subsample': 0.9193661622887357, 'reg_alpha': 1.0085528642963655, 'reg_lambda': 3.7034168467053896}. Best is trial 0 with value: 0.6749699650502403.
[I 2025-02-16 01:07:04,937] Trial 2 finished with value: 0.6878194626474443 and parameters: {'n_estimators': 5422, 'max_depth': 249, 'learning_rate': 0.004

Best trial:
  Value: 0.7080548274355616
  Params: 
    n_estimators: 4428
    max_depth: 56
    learning_rate: 9.61231614378315e-05
    num_leaves: 1077
    min_child_samples: 59
    subsample: 0.6597060374327504
    reg_alpha: 6.184992419868237
    reg_lambda: 0.052571359776606284


.